<a href="https://colab.research.google.com/github/Puneet-Bajaj-IITM/IOTA-MODELS/blob/main/Server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ipfshttpclient iota cryptography aiohttp nio flask

In [ ]:
import json
import ipfshttpclient
from iota import Iota, ProposedTransaction, Address, Tag, TryteString

# IPFS Client
ipfs_client = ipfshttpclient.connect()

# IOTA Client (Replace with your IOTA node URL)
api = Iota("https://nodes.devnet.iota.org:443", seed="YOUR_SEED_HERE")

def upload_model_to_public_iota(model_name, weights_path, config_path):
    # Upload to IPFS
    weights_cid = ipfs_client.add(weights_path)['Hash']
    config_cid = ipfs_client.add(config_path)['Hash']

    # Metadata for the model
    metadata = {
        "model_name": model_name,
        "weights_cid": weights_cid,
        "config_cid": config_cid
    }

    # Save metadata to IOTA
    metadata_trytes = TryteString.from_unicode(json.dumps(metadata))
    [address] = api.get_new_addresses(count=1)['addresses']

    txn = ProposedTransaction(
        address=Address(address),
        value=0,
        tag=Tag(b"MODEL_PUBLIC_UPLOAD"),
        message=metadata_trytes
    )
    api.send_transfer(transfers=[txn])

    return {
        "transaction_hash": txn.hash,
        "weights_cid": weights_cid,
        "config_cid": config_cid
    }


In [ ]:
import json
import ipfshttpclient


# Registry File (for IPFS-based registry)
REGISTRY_FILE = "model_registry.json"

# Initialize an empty registry
def initialize_registry():
    registry = {}
    with open(REGISTRY_FILE, "w") as f:
        json.dump(registry, f)
    cid = ipfs_client.add(REGISTRY_FILE)["Hash"]
    print(f"Initialized registry with CID: {cid}")
    return cid

def update_registry(model_name, transaction_hash):
    # Fetch the existing registry
    ipfs_client.get(REGISTRY_FILE, target=REGISTRY_FILE)
    with open(REGISTRY_FILE, "r") as f:
        registry = json.load(f)

    # Add the new model
    registry[model_name] = transaction_hash

    # Update the registry on IPFS
    with open(REGISTRY_FILE, "w") as f:
        json.dump(registry, f)
    cid = ipfs_client.add(REGISTRY_FILE)["Hash"]
    print(f"Updated registry with CID: {cid}")
    return cid




In [ ]:
@app.route("/fetch_model", methods=["GET"])
def fetch_model():
    """Fetch model metadata by name or transaction hash."""
    model_name = request.args.get("model_name")
    transaction_hash = request.args.get("transaction_hash")

    if not model_name and not transaction_hash:
        return jsonify({"error": "Provide either model_name or transaction_hash"}), 400

    try:
        # Resolve transaction hash from registry if model_name is provided
        if model_name:
            ipfs_client.get(REGISTRY_FILE, target=REGISTRY_FILE)
            with open(REGISTRY_FILE, "r") as f:
                registry = json.load(f)

            if model_name not in registry:
                return jsonify({"error": "Model name not found in registry"}), 404

            transaction_hash = registry[model_name]

        # Fetch metadata from IOTA
        transaction = iota_client.get_trytes([transaction_hash])["trytes"][0]
        metadata = json.loads(TryteString(transaction.signature_message_fragment).decode())

        model_name = metadata["model_name"]
        weights_cid = metadata["weights_cid"]
        config_cid = metadata["config_cid"]

        # Download from IPFS
        weights_path = f"{model_name}_weights.pth"
        config_path = f"{model_name}_config.json"

        ipfs_client.get(weights_cid, target=weights_path)
        ipfs_client.get(config_cid, target=config_path)

        return jsonify({
            "model_name": model_name,
            "weights_path": weights_path,
            "config_path": config_path,
            "weights_cid": weights_cid,
            "config_cid": config_cid,
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
import asyncio
from nio import AsyncClient

async def propose_update(room_id, model_name, transaction_hash, new_weights_cid, new_config_cid):
    client = AsyncClient("https://matrix.org", "@username:matrix.org")
    await client.login("PASSWORD")

    message = (
        f"Proposal to update model '{model_name}':\n"
        f"Transaction Hash: {transaction_hash}\n"
        f"New Weights CID: {new_weights_cid}\n"
        f"New Config CID: {new_config_cid}\n"
        "Vote with 'yes' or 'no'."
    )

    await client.room_send(
        room_id=room_id,
        message_type="m.room.message",
        content={"msgtype": "m.text", "body": message}
    )
    await client.logout()


In [ ]:
@app.route("/add_model", methods=["POST"])
def add_model():
    """Add a new model and update the registry."""
    try:
        data = request.json
        model_name = data["model_name"]
        weights_path = data["weights_path"]
        config_path = data["config_path"]

        # Upload to IPFS
        weights_cid = ipfs_client.add(weights_path)["Hash"]
        config_cid = ipfs_client.add(config_path)["Hash"]

        # Save metadata to IOTA
        metadata = {
            "model_name": model_name,
            "weights_cid": weights_cid,
            "config_cid": config_cid,
        }
        metadata_trytes = TryteString.from_unicode(json.dumps(metadata))
        [address] = iota_client.get_new_addresses(count=1)["addresses"]

        txn = ProposedTransaction(
            address=Address(address),
            value=0,
            tag=Tag(b"MODEL_ADD"),
            message=metadata_trytes,
        )
        result = iota_client.send_transfer(transfers=[txn])

        # Update the registry with the new model
        transaction_hash = result["bundle"][0].hash
        registry_cid = update_registry(model_name, str(transaction_hash))

        return jsonify({
            "transaction_hash": transaction_hash,
            "weights_cid": weights_cid,
            "config_cid": config_cid,
            "registry_cid": registry_cid,
        })
    except Exception as e:
        return jsonify({"error": str(e)}), 500


In [ ]:
curl -X POST "http://localhost:5000/add_model" \
     -H "Content-Type: application/json" \
     -d '{
         "model_name": "example_model",
         "weights_path": "example_weights.pth",
         "config_path": "example_config.json"
     }'


In [ ]:
curl "http://localhost:5000/fetch_model?model_name=example_model"
